# FABRIC Example: Map of Resources


## Configure the Environment

This process may take a moment, as the folium mapping package must be installed to the kernel.

In [1]:
import os
os.system('conda install folium')
from fabrictestbed.slice_manager import SliceManager, Status
import json

credmgr_host = os.environ['FABRIC_CREDMGR_HOST']
orchestrator_host = os.environ['FABRIC_ORCHESTRATOR_HOST']
print(f"CM Host: {credmgr_host} Orchestrator Host: {orchestrator_host}")

#Configure SSH Key
ssh_key = None
with open ("/home/fabric/.ssh/id_rsa.pub", "r") as myfile:
    ssh_key=myfile.read().strip()

CM Host: beta-2.fabric-testbed.net Orchestrator Host: beta-7.fabric-testbed.net


### Save the Initial Refresh Token

In [2]:
#Retrieve or set the refresh token (exprires 24 hours after login)
not_found=False
fabric_refresh_token=None
%store -r fabric_refresh_token

if fabric_refresh_token is None:
    fabric_refresh_token=os.environ['CILOGON_REFRESH_TOKEN']
    %store fabric_refresh_token
print("Fabric Refresh Token {}".format(fabric_refresh_token))
print("CILOGON_REFRESH_TOKEN environment variable: {}".format(os.environ['CILOGON_REFRESH_TOKEN']))

Fabric Refresh Token NB2HI4DTHIXS6Y3JNRXWO33OFZXXEZZPN5QXK5DIGIXTGMTFGU4TQNJYMRRWGYRRMZSDGYRYGE4GMMBVMZQTIZBSMM4DOMJ7OR4XAZJ5OJSWM4TFONUFI33LMVXCM5DTHUYTMMRZGMYDSOJXG4ZDKOBGOZSXE43JN5XD25RSFYYCM3DJMZSXI2LNMU6TQNRUGAYDAMBQ
CILOGON_REFRESH_TOKEN environment variable: NB2HI4DTHIXS6Y3JNRXWO33OFZXXEZZPN5QXK5DIGIXTCYLFMVRDCNZQMRRWIYTFHE2DEZRWGAYDCYRSGU2WENJUMM2TIMJ7OR4XAZJ5OJSWM4TFONUFI33LMVXCM5DTHUYTMMRZGMYDSOBWHAYTQMBGOZSXE43JN5XD25RSFYYCM3DJMZSXI2LNMU6TQNRUGAYDAMBQ


## Create and Initialize Slice Manager Object
Users can request tokens with different Project and Scopes by altering `project_name` and `scope` parameters in the refresh call below.

In [3]:
slice_manager = SliceManager(oc_host = orchestrator_host, cm_host = credmgr_host, project_name = 'all', scope = 'all')
slice_manager.initialize()

### Refresh the ID Token

ID Tokens expire one hour after refresh. 

In [4]:
try:
    id_token, refresh_token = slice_manager.refresh_tokens()
except Exception as e:
    print("Exception occurred while getting tokens:{}".format(e))

fabric_refresh_token=slice_manager.get_refresh_token()
print()
print("New Refresh Token: {}".format(fabric_refresh_token))
print()
print("Stored new Refresh Token")
%store fabric_refresh_token


New Refresh Token: NB2HI4DTHIXS6Y3JNRXWO33OFZXXEZZPN5QXK5DIGIXTEM3EGBRWKM3BGY3GGNJUMI4TOZBUGU3TQMBWMJSWCMDFGBSDIOB7OR4XAZJ5OJSWM4TFONUFI33LMVXCM5DTHUYTMMRZGMYTAMJUGY2DSMBGOZSXE43JN5XD25RSFYYCM3DJMZSXI2LNMU6TQNRUGAYDAMBQ

Stored new Refresh Token
Stored 'fabric_refresh_token' (str)


### Query Slices

In [5]:
status, slices = slice_manager.slices()

print("Response Status {}".format(status))
if status == Status.OK:
    print("Slices {}".format(slices))
else:
    print(f"Failure: {slices}")

Response Status Status.OK
Slices [{
    "graph_id": "cea05cc8-e145-4972-9afd-bcfd2f3338a6",
    "lease_end": "2021-08-19 18:06:29",
    "slice_id": "0b06366f-7fbd-42ff-b497-87c8ea4df85d",
    "slice_name": "Slice 2",
    "slice_state": "Configuring"
}, {
    "graph_id": "5c29e50d-d1ac-42f7-937e-f05bcbeea4d1",
    "lease_end": "2021-08-19 18:06:29",
    "slice_id": "0011f0e7-d3c6-484f-bd4a-183271d32515",
    "slice_name": "Slice 3",
    "slice_state": "Configuring"
}, {
    "graph_id": "b0359555-7d51-40c9-905e-553871126f52",
    "lease_end": "2021-08-19 18:06:29",
    "slice_id": "3c884e33-1f40-421a-89b9-586f81a2e019",
    "slice_name": "Slice 1",
    "slice_state": "StableOK"
}, {
    "graph_id": "6a8e14af-d476-4498-ad7e-511b2ef627d1",
    "lease_end": "2021-08-19 18:06:29",
    "slice_id": "e3ae8b3d-115b-4e66-a035-a6da986e171a",
    "slice_name": "Slice 4 (l2bridge-sriov)",
    "slice_state": "Dead"
}]


### Get Slice Information, Query Slivers From Slices

In [6]:
slice_ids = [slice.slice_id for slice in slices]
slice_names = [slice.slice_name for slice in slices]
slice_lease_ends = [slice.lease_end for slice in slices]
slice_graph_ids = [slice.graph_id for slice in slices]
clean_slice_lease_ends = []

months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
for sle in slice_lease_ends:
    year = sle[:4]
    month = months[int(sle[5:7]) - 1]
    day = sle[8:10]
    if day[0] == '0':
        day = day[1]
    hour = sle[11:13]
    ampm = "AM" if int(hour) <= 12 else "PM"
    if hour[0] == '0':
        hour = hour[1]
    if int(hour) > 12:
        hour = str(int(hour) - 12)
    minute = sle[14:16]
    second = sle[17:19]
    date = month + ' ' + day + ', ' + year + f' [{hour}:{minute}:{second} {ampm}]'
    clean_slice_lease_ends.append(date)
    

for slice_id in slice_ids:
    status, slivers = slice_manager.slivers(slice_id = slice_id)
    
    print(f'\nSlice Name: {slice_names[slice_ids.index(slice_id)]}\n')
    print("Response Status {}".format(status))
    if status == Status.OK:
        print("Slivers {}".format(slivers))
    else:
        print(f"Failure: {slivers}")


Slice Name: Slice 2

Response Status Status.OK
Slivers [{
    "allocated_capacities": "{\"core\": 4, \"disk\": 10, \"ram\": 16}",
    "allocated_labels": "{\"instance\": \"instance-00000130\", \"instance_parent\": \"lbnl-w2.fabric-testbed.net\"}",
    "capacities": "{\"core\": 3, \"disk\": 10, \"ram\": 6}",
    "capacity_hints": "{\"instance_type\": \"fabric.c4.m16.d10\"}",
    "graph_node_id": "218bb69e-703a-495c-bd59-7b1d9ab5f8e9",
    "join_state": "NoJoin",
    "lease_end": "2021-08-19 18:06:29",
    "management_ip": "198.129.61.39",
    "name": "S2Node1",
    "pending_state": "None_",
    "reservation_id": "0003761b-795e-4422-98d8-d6c548b5ce65",
    "reservation_state": "Active",
    "resource_type": "VM",
    "site": "LBNL",
    "slice_id": "0b06366f-7fbd-42ff-b497-87c8ea4df85d"
}, {
    "allocated_capacities": "{\"core\": 4, \"disk\": 10, \"ram\": 16}",
    "allocated_labels": "{\"instance\": \"instance-00000132\", \"instance_parent\": \"lbnl-w2.fabric-testbed.net\"}",
    "cap

### Create and Save Map

- Click 'Generate' to generate map
- Click 'Clear' to clear screen
- Click check boxes to switch between slices (or 'View All' to view all)
- Click markers to view details about individual resources
- Map saves to working directory as 'map.html'

In [18]:
status, advertised_topology = slice_manager.resources()
site_dict = {}

for t in advertised_topology.sites.values():
    name = t.name 
    loc_info = t.get_property("location")
    if loc_info is not None:
        loc = loc_info.to_latlon()
        site_dict[name] = loc
        
import folium
from folium.plugins import MarkerCluster
from folium.features import DivIcon
from folium.raster_layers import WmsTileLayer
import branca
from branca.element import Template, MacroElement
import ipywidgets as widgets
from IPython.display import clear_output
from math import sqrt

template = """
        {% macro html(this, kwargs) %}

        <!doctype html>
        <html lang="en">
        <head>
          <meta charset="utf-8">
          <meta name="viewport" content="width=device-width, initial-scale=1">
          <title>jQuery UI Draggable - Default functionality</title>
          <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

          <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
          <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>

          <script>
          $( function() {
            $( "#maplegend" ).draggable({
                            start: function (event, ui) {
                                $(this).css({
                                    right: "auto",
                                    top: "auto",
                                    bottom: "auto"
                                });
                            }
                        });
        });

          </script>
        </head>
        <body>


        <div id='maplegend' class='maplegend' 
            style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
             border-radius:6px; padding: 10px; font-size:15px;font-family: Arial; right: 20px; bottom: 20px;'>

        <div class='legend-title'>Legend</div>
        <div class='legend-scale'>
          <ul class='legend-labels'>
            <li><span style='background:green;opacity:0.8;'></span>Active</li>
            <li><span style='background:orange;opacity:0.8;'></span>Closing</li>
            <li><span style='background:red;opacity:0.8;'></span>Closed</li>

          </ul>
        </div>
        </div>

        </body>
        </html>

        <style type='text/css'>
          .maplegend .legend-title {
            text-align: left;
            margin-bottom: 5px;
            font-weight: bold;
            font-size: 90%;
            }
          .maplegend .legend-scale ul {
            margin: 0;
            margin-bottom: 5px;
            padding: 0;
            float: left;
            list-style: none;
            }
          .maplegend .legend-scale ul li {
            font-size: 80%;
            list-style: none;
            margin-left: 0;
            line-height: 18px;
            margin-bottom: 2px;
            }
          .maplegend ul.legend-labels li span {
            display: block;
            float: left;
            height: 16px;
            width: 30px;
            margin-right: 5px;
            margin-left: 0;
            border: 1px solid #999;
            }
          .maplegend .legend-source {
            font-size: 80%;
            color: #777;
            clear: both;
            }
          .maplegend a {
            color: #777;
            }

        </style>
        {% endmacro %}
        """

url = ('http://services.arcgisonline.com/arcgis/rest/services/World_Street_Map' + '/MapServer/tile/{z}/{y}/{x}')

def generate(b):
    with output:
        clear_output()
        base = folium.Map(location = (38.12480976137421, -95.7129), zoom_start = 4.2, zoom_control = False, scrollWheelZoom = False, dragging = False, max_bounds = True)
        WmsTileLayer(url = url, layers = None, name = 'ESRI World Street Map', attr = 'ESRI World Street Map', control = False).add_to(base)

        for slice_id in slice_ids:
            status, slivers = slice_manager.slivers(slice_id = slice_id)
            status, t = slice_manager.get_slice(slice_id = slice_id)

            labels = {}
            for sliver in slivers:
                status, sliver_status = slice_manager.sliver_status(slice_id = slice_id, sliver_id = sliver.reservation_id)
                if sliver_status.resource_type == "VM":
                    name = sliver_status.name
                    site = sliver_status.site
                    capacities = sliver_status.capacities
                    components = [t.nodes[name].get_property('image_ref'), t.nodes[name].get_property('image_type')]
                    state = sliver_status.reservation_state
                    info = (site, capacities, components, state)
                    labels[name] = info

            locations = []
            popups = []
            icons = []

            for label in labels:
                locations.append(site_dict[labels[label][0]])
                popups.append(folium.Popup(branca.element.IFrame(html = f'<strong style="font-family: Arial">Sliver: {label} [{labels[label][3]}]</strong><div style="font-family: Arial; font-size: 10pt"><br>Site: {labels[label][0]}<br>Capacities: {labels[label][1]}<br>Image Ref: {labels[label][2][0]}<br>Image Type: {labels[label][2][1]}', width = 350, height = 130), max_width = 360))
                color = None
                if labels[label][3] == 'Active':
                    color = 'green'
                elif labels[label][3] == 'Closing':
                    color = 'orange'
                else:
                    color = 'red'
                icons.append(folium.Icon(color = color, icon = 'server', prefix = 'fa'))

            cluster = MarkerCluster(name = slice_names[slice_ids.index(slice_id)], 
                                    locations = locations, 
                                    popups = popups, 
                                    icons = icons,
                                    show = False)
            folium.map.Marker((54, -110), 
                              icon = DivIcon(icon_size = (300, 80),
                                             icon_anchor = (0, 0),
                                             html = f'<div style="font-size: 15pt;font-family: Arial;text-align: center;border: 1pt solid lightgray;background: ghostwhite">{slice_names[slice_ids.index(slice_id)]}<h3 style="font-size: 10pt;font-family: Arial">Lease End: {clean_slice_lease_ends[slice_ids.index(slice_id)]}')).add_to(cluster)
            for sliver in slivers:
                status, sliver_status = slice_manager.sliver_status(slice_id = slice_id, sliver_id = sliver.reservation_id)
                if sliver_status.resource_type != "VM":
                    name = sliver_status.name
                    site = sliver_status.site
                    state = sliver_status.reservation_state
                    link = None
                    if state == 'Active':
                        link = 'https://www.pngmart.com/files/4/Green-Light-PNG-Transparent-Image.png'
                    elif state == 'Closing':
                        link = 'https://www.clipartmax.com/png/middle/178-1787887_light-png-orange-light-png.png'
                    else:
                        link = 'https://toppng.com/uploads/preview/red-glowing-eyes-png-circle-11562910600ypa0i4jelr.png'
                    icon = folium.features.CustomIcon(link, icon_size = (30, 30))
                    popup = folium.Popup(branca.element.IFrame(html = f'<strong style="font-family: Arial">L2Bridge: {name} [{state}]</strong><div style="font-family: Arial; font-size: 10pt"><br>Site: {site}', width = 350, height = 130), max_width = 360)
                    folium.map.Marker(site_dict[site], name = name, icon = icon, popup = popup).add_to(cluster)
                
            cluster.add_to(base)
            
        labels = {}
        for slice_id in slice_ids:
            status, slivers = slice_manager.slivers(slice_id = slice_id)
            status, t = slice_manager.get_slice(slice_id = slice_id)

            for sliver in slivers:
                status, sliver_status = slice_manager.sliver_status(slice_id = slice_id, sliver_id = sliver.reservation_id)
                if sliver_status.resource_type == "VM":
                    title = sliver_status.name + sliver_status.lease_end
                    name = sliver_status.name
                    site = sliver_status.site
                    capacities = sliver_status.capacities
                    components = [t.nodes[name].get_property('image_ref'), t.nodes[name].get_property('image_type')]
                    state = sliver_status.reservation_state
                    end = clean_slice_lease_ends[slice_ids.index(slice_id)]
                    parent = slice_names[slice_ids.index(slice_id)]
                    info = (site, capacities, components, state, end, name, parent)
                    labels[title] = info

        locations = []
        popups = []
        icons = []

        for label in labels:
            locations.append(site_dict[labels[label][0]])
            popups.append(folium.Popup(branca.element.IFrame(html = f'<strong style="font-family: Arial">Sliver: {labels[label][-2]} [{labels[label][3]}]</strong><br><div style="font-family: Arial; font-size: 10pt">Parent Slice: {labels[label][-1]}<br>Lease End: {labels[label][-3]}<br><br>Site: {labels[label][0]}<br>Capacities: {labels[label][1]}<br>Image Ref: {labels[label][2][0]}<br>Image Type: {labels[label][2][1]}', width = 350, height = 130), max_width = 360))
            color = None
            if labels[label][3] == 'Active':
                color = 'green'
            elif labels[label][3] == 'Closing':
                color = 'orange'
            else:
                color = 'red'
            icons.append(folium.Icon(color = color, icon = 'server', prefix = 'fa'))

        
        cluster = MarkerCluster(name = 'View All', 
                                locations = locations, 
                                popups = popups, 
                                icons = icons,
                                show = False)
        
        for slice_id in slice_ids:
            status, slivers = slice_manager.slivers(slice_id = slice_id)
            for sliver in slivers:
                status, sliver_status = slice_manager.sliver_status(slice_id = slice_id, sliver_id = sliver.reservation_id)
                if sliver_status.resource_type != "VM":
                    name = sliver_status.name
                    site = sliver_status.site
                    state = sliver_status.reservation_state
                    parent = slice_names[slice_ids.index(slice_id)]
                    end = clean_slice_lease_ends[slice_ids.index(slice_id)]
                    link = None
                    if state == 'Active':
                        link = 'https://www.pngmart.com/files/4/Green-Light-PNG-Transparent-Image.png'
                    elif state == 'Closing':
                        link = 'https://www.clipartmax.com/png/middle/178-1787887_light-png-orange-light-png.png'
                    else:
                        link = 'https://toppng.com/uploads/preview/red-glowing-eyes-png-circle-11562910600ypa0i4jelr.png'
                    icon = folium.features.CustomIcon(link, icon_size = (30, 30))
                    popup = folium.Popup(branca.element.IFrame(html = f'<strong style="font-family: Arial">L2Bridge: {name} [{state}]</strong><div style="font-family: Arial; font-size: 10pt">Parent Slice: {parent}<br>Lease End: {end}<br><br>Site: {site}', width = 350, height = 130), max_width = 360)
                    folium.map.Marker(site_dict[site], name = name, icon = icon, popup = popup).add_to(cluster)

        cluster.add_to(base)
            
        folium.LayerControl(collapse = False).add_to(base)  
        macro = MacroElement()
        macro._template = Template(template)
        base.get_root().add_child(macro)
        base.save('map.html')
        display(base)
        
def clear(b):
    with output:
        clear_output()
        
generate_button = widgets.Button(description = 'Generate Map', disabled = False, tooltip = 'Click to generate map')
generate_button.style.button_color = '#27aae1'
clear_button = widgets.Button(description = 'Clear Screen', disabled = False, tooltip = 'Click to clear screen')
clear_button.style.button_color = '#f26522'
generate_button.on_click(generate)
clear_button.on_click(clear)
output = widgets.Output()

buttons = widgets.HBox([generate_button, clear_button])
widgets.VBox([buttons, output])